Extrae el % de bateria del comentario y lo pega en la columna de bateria

In [8]:
import pandas as pd
import re
from datetime import datetime

In [9]:
elemento_a_buscar = "iphone"

carpeta = "4_download_description"
fecha = datetime.now().strftime("%Y%m%d")
fecha = "20250106"

In [10]:

nombre_archivo = fecha + '_' + elemento_a_buscar
df_original = pd.read_csv(carpeta + '/' + nombre_archivo + '.csv')
df_original.drop('Unnamed: 0', axis=1, inplace=True)
df_original.head()

,id,fecha,gen,mod,memoria,bateria,precio,estado,nombre,scrap_locat,dist_scrap_locat,reservado,url,tiene_color,tiene_emojis,tiene_revisado,comentario
0,1033637533,20250106,14,basic,128,NaN,"615,37",good,apple iphone 14 128gb amarillo - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-14-1...,True,True,True,Marca: Apple\nModelo: iPhone 14\nCapacidad de ...
1,1033637778,20250106,15,basic,128,NaN,"857,00",good,apple iphone 15 128gb verde - revisado,madrid,60,False,https://es.wallapop.com/item/apple-iphone-15-1...,True,True,True,Marca: Apple\nModelo: iPhone 15\nCapacidad de ...
2,1056720567,20250106,14,plus,NaN,NaN,"490,00",as_good_as_new,iphone 14 plus,madrid,60,False,https://es.wallapop.com/item/iphone-14-plus-10...,False,False,False,"Detalles del producto\nIphone 14 plus, no tien..."
3,1078161119,20250106,15,pro max,NaN,NaN,"1095,00",as_good_as_new,iphone 15 pro max !,madrid,60,False,https://es.wallapop.com/item/iphone-13-pro-en-...,False,True,False,Detalles del producto\niPhone 15 pro Max De 51...
4,1079999158,20250106,16,pro,256,NaN,"1150,00",as_good_as_new,iphone 16 pro 256gb,madrid,60,False,https://es.wallapop.com/item/iphone-16-pro-256...,False,False,False,Detalles del producto\nVendo iphone 16 pro de ...


In [11]:
df = df_original.copy()


In [12]:
def encontrar_bateria(texto: str):
    # Verificar si el texto es NaN o similar
    if pd.isna(texto):
        return None
    # Buscar números seguidos por %
    numeros_con_porcentaje = re.findall(r'\b(8[0-9]|9[0-9]|100)\b%', texto)
    # Si encuentra números con %, devolver el primero
    if numeros_con_porcentaje:
        return numeros_con_porcentaje[0] 
    # Si no hay % buscar solo los números
    numeros = re.findall(r'\b(8[0-9]|9[0-9]|100)\b', texto)
    # Si encuentra números, devolver el primero
    if numeros:
        return numeros[0]
    # Si no encuentra ningún número, devolver None
    return None

In [13]:
def encontrar_memoria(texto: str) -> str:
    if pd.isna(texto):
        return None
    texto = str(texto)
    # Buscar memorias seguidas por Mb, MB, mb, T o t
    memorias_con_unidades = re.findall(r'\b(128|256|512|1[Tt])\b\s*(Mb|MB|mb|[Tt])', texto)
    if memorias_con_unidades:
        memoria, unidad = memorias_con_unidades[0]
        # Si es 1T o 1t, convertirlo a '1TB'
        if memoria.lower() == '1t':
            return '1TB'
        return memoria + unidad  # Devolver memoria seguida de la unidad
    
    # Si no encuentra memorias con unidades, buscar solo las capacidades de memoria
    memorias = re.findall(r'\b(128|256|512|1[Tt])\b', texto)
    if memorias:
        # Convertir 1T o 1t a '1TB'
        if memorias[0].lower() == '1t':
            return '1TB'
        return memorias[0]
    return None
    
def es_tienda(texto: str) -> bool:
    if pd.isna(texto):
        return None
    words_to_find = ('reacondicionado', 'tienda')
    for word in words_to_find:
        if word in texto:
            return True 
    return False 

def tiene_garantia(texto: str):
    if pd.isna(texto):
        return None
    words_to_find = ('garantia', 'care', 'factura')
    for word in words_to_find:
        if word in texto:
            return True 
    return False 

In [14]:

missing_values = df_original.isnull().sum()
print(missing_values)
df['bateria'] = df_original['comentario'].apply(encontrar_bateria)
df['memoria'] = df_original['comentario'].apply(encontrar_memoria)
df['tienda'] = df_original['comentario'].apply(es_tienda)
df['garantia'] = df_original['comentario'].apply(tiene_garantia)

missing_values = df.isnull().sum()
print(missing_values)


id                     0
fecha                  0
gen                    0
mod                    0
memoria              382
bateria             1087
precio                 0
estado                 0
nombre                 0
scrap_locat            0
dist_scrap_locat       0
reservado              0
url                    0
tiene_color            0
tiene_emojis           0
tiene_revisado         0
comentario           111
dtype: int64


id                    0
fecha                 0
gen                   0
mod                   0
memoria             283
bateria             482
precio                0
estado                0
nombre                0
scrap_locat           0
dist_scrap_locat      0
reservado             0
url                   0
tiene_color           0
tiene_emojis          0
tiene_revisado        0
comentario          111
tienda              111
garantia            111
dtype: int64


In [16]:
df['tienda'].describe()

count      1049
unique        2
top       False
freq        859
Name: tienda, dtype: object